In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
id_information = pd.read_csv('id_information_mmsr.tsv', sep='\t')
id_information = id_information.sort_values('id', ascending=True)
id_genres = pd.read_csv('id_genres_mmsr.tsv', sep='\t')
id_genres = id_genres.sort_values('id', ascending=True)
id_bert = pd.read_csv('id_bert_mmsr.tsv', sep='\t')
id_bert = id_bert.sort_values('id', ascending=True)
id_lyrics_tfidf = pd.read_csv('id_lyrics_tf-idf_mmsr.tsv', sep='\t')
id_lyrics_tfidf = id_lyrics_tfidf.sort_values('id', ascending=True)
id_lyrics_word2vec = pd.read_csv('id_lyrics_word2vec_mmsr.tsv', sep='\t')
id_lyrics_word2vec = id_lyrics_word2vec.sort_values('id', ascending=True)

In [128]:
query_id = 'w1DNGoZLFFnLQO7L'


In [129]:
#function that returns conine similarity between a rowvector(1,c) and a matrix of rowvectors (r,c)
#returns columnvector (r,1)

def cosine_similarities(x, Y):
    nx = np.linalg.norm(x)
    nY = np.linalg.norm(Y, axis=1, keepdims=True)
    DotP = np.dot(x, Y.T).T
    
    return DotP/((nx*nY)+1e-10) 

In [130]:
tfidf_subset = id_lyrics_tfidf
tfidf_query = id_lyrics_tfidf.loc[id_lyrics_tfidf['id']==query_id]
tfidf_query = np.array(tfidf_query.drop(columns=['id']))
subset_ids = tfidf_subset[['id']].copy()

tfidf_no_id = np.array(tfidf_subset.drop(columns=['id']))
cosine = cosine_similarities(tfidf_query, tfidf_no_id)
subset_ids.insert(1,'cosine_ifidf', cosine)


bert_subset = id_bert
bert_query = id_bert.loc[id_bert['id']==query_id]
bert_query = np.array(bert_query.drop(columns=['id']))

bert_no_id = np.array(bert_subset.drop(columns=['id']))
cosine = cosine_similarities(bert_query, bert_no_id)
subset_ids.insert(1,'cosine_bert', cosine)


word2vec_subset = id_lyrics_word2vec
word2vec_query = id_lyrics_word2vec.loc[id_lyrics_word2vec['id']==query_id]
word2vec_query = np.array(word2vec_query.drop(columns=['id']))

word2vec_no_id = np.array(word2vec_subset.drop(columns=['id']))
cosine = cosine_similarities(word2vec_query, word2vec_no_id)
subset_ids.insert(1,'cosine_word2vec', cosine)
subset_ids_without_id = subset_ids.drop(columns=['id'])

avg_cosin = subset_ids_without_id.mean(axis=1)
subset_ids.insert(1, 'avg_cosin', avg_cosin)
subset_ids

,id,avg_cosin,cosine_word2vec,cosine_bert,cosine_ifidf
26006,0009fFIM1eYThaPg,0.419071,0.691502,0.501530,0.064180
9041,0010xmHR6UICBOYT,0.399592,0.826663,0.273172,0.098940
64166,002Jyd0vN4HyCpqL,0.355038,0.786899,0.215148,0.063066
10342,006TYKNjNxWjfKjy,0.367660,0.699043,0.374296,0.029640
40355,007LIJOPQ4Sb98qV,0.427915,0.775757,0.482549,0.025438
...,...,...,...,...,...
26450,zzx8CWdM7qkxKQpC,0.469286,0.876056,0.387474,0.144327
8970,zzyb5LvKJTWLVnrk,0.352900,0.732167,0.298515,0.028019
73934,zzz0n04uuTUA7fNh,0.721138,0.951979,0.782646,0.428790
69128,zzznMjZAKnJJXQSj,0.577054,0.924749,0.609638,0.196774


In [131]:
tfidf_query[tfidf_query != 0.0]=1
tfidf_no_id[tfidf_no_id != 0.0]=1
scores = []
for row in tqdm(tfidf_no_id):
    jaccard = jaccard_score(tfidf_query[0], row)
    scores.append(jaccard)
    
#AUSKOMMENTIEREN bei neustart!!!!!!
#subset_ids.drop(columns=['jaccard_ifidf'])
subset_ids.insert(1,'jaccard_ifidf', scores)

subset_ids

100%|███████████████████████████████████████████████████████████████████████████| 76115/76115 [02:10<00:00, 584.42it/s]


,id,jaccard_ifidf,avg_cosin,cosine_word2vec,cosine_bert,cosine_ifidf
26006,0009fFIM1eYThaPg,0.152381,0.419071,0.691502,0.501530,0.064180
9041,0010xmHR6UICBOYT,0.105960,0.399592,0.826663,0.273172,0.098940
64166,002Jyd0vN4HyCpqL,0.092437,0.355038,0.786899,0.215148,0.063066
10342,006TYKNjNxWjfKjy,0.052632,0.367660,0.699043,0.374296,0.029640
40355,007LIJOPQ4Sb98qV,0.055046,0.427915,0.775757,0.482549,0.025438
...,...,...,...,...,...,...
26450,zzx8CWdM7qkxKQpC,0.135338,0.469286,0.876056,0.387474,0.144327
8970,zzyb5LvKJTWLVnrk,0.084211,0.352900,0.732167,0.298515,0.028019
73934,zzz0n04uuTUA7fNh,0.254237,0.721138,0.951979,0.782646,0.428790
69128,zzznMjZAKnJJXQSj,0.152542,0.577054,0.924749,0.609638,0.196774


In [132]:
subset_ids_new = subset_ids.drop(columns=['avg_cosin'])
subset_ids_without_id = subset_ids_new.drop(columns=['id'])
avg_sim = subset_ids_without_id.mean(axis=1)
subset_ids_new.insert(1, 'avg_similarity', avg_sim)
subset_ids_new

,id,avg_similarity,jaccard_ifidf,cosine_word2vec,cosine_bert,cosine_ifidf
26006,0009fFIM1eYThaPg,0.352398,0.152381,0.691502,0.501530,0.064180
9041,0010xmHR6UICBOYT,0.326184,0.105960,0.826663,0.273172,0.098940
64166,002Jyd0vN4HyCpqL,0.289387,0.092437,0.786899,0.215148,0.063066
10342,006TYKNjNxWjfKjy,0.288903,0.052632,0.699043,0.374296,0.029640
40355,007LIJOPQ4Sb98qV,0.334698,0.055046,0.775757,0.482549,0.025438
...,...,...,...,...,...,...
26450,zzx8CWdM7qkxKQpC,0.385799,0.135338,0.876056,0.387474,0.144327
8970,zzyb5LvKJTWLVnrk,0.285728,0.084211,0.732167,0.298515,0.028019
73934,zzz0n04uuTUA7fNh,0.604413,0.254237,0.951979,0.782646,0.428790
69128,zzznMjZAKnJJXQSj,0.470926,0.152542,0.924749,0.609638,0.196774


In [133]:
subset_ids= subset_ids_new.sort_values('avg_similarity', ascending=False)
subset_ids.head(100)

,id,avg_similarity,jaccard_ifidf,cosine_word2vec,cosine_bert,cosine_ifidf
35956,w1DNGoZLFFnLQO7L,1.000000,1.000000,1.000000,1.000000,1.000000
33593,giW3SogXe6AZeWUP,0.620531,0.228070,0.938577,0.814961,0.500515
24358,FFzYMGfA323nt6nv,0.617966,0.221154,0.934981,0.786026,0.529704
59551,lTXc4hPGzIvKncHE,0.616635,0.260870,0.945136,0.817853,0.442681
39719,v3Oham7Iybn77VsR,0.605100,0.287129,0.933840,0.751837,0.447594
...,...,...,...,...,...,...
43205,TGW1SdawoRO9ZZ0R,0.564930,0.254098,0.940616,0.801997,0.263008
29782,exX6CsmLe6uh5jjf,0.564839,0.151079,0.937927,0.598285,0.572066
37905,pYRd0RiBFH73TC8V,0.564795,0.170213,0.942511,0.711936,0.434520
48341,bU2Ot5VcJbb6V2uE,0.563461,0.233645,0.927522,0.738664,0.354013


In [134]:
similarity_df = pd.read_csv("similaritys_already_searched.tsv", sep = "\t", index_col=[0])
sorted_ids = np.array(subset_ids['id'])

print(sorted_ids)
if query_id not in similarity_df.columns:
    similarity_df.insert(0, query_id, sorted_ids)
    
similarity_df


['w1DNGoZLFFnLQO7L' 'giW3SogXe6AZeWUP' 'FFzYMGfA323nt6nv' ...
 'pq3XcYiHrt5tlDtB' 'SLMm4hSPBChYpw4q' 'UDYzAIMYVZzFkeDR']


,w1DNGoZLFFnLQO7L,tSgGdR5fSA3UOD9T
0,w1DNGoZLFFnLQO7L,tSgGdR5fSA3UOD9T
1,giW3SogXe6AZeWUP,3Y9Uokc2meAZVARt
2,FFzYMGfA323nt6nv,7zLJQ1oFhvWLDdjc
3,lTXc4hPGzIvKncHE,6l03X9F1uc0LEQnJ
4,v3Oham7Iybn77VsR,w1DNGoZLFFnLQO7L
...,...,...
76110,6KgU4jXWiWWbrh8a,rQ8RT1Y3coPMwk6w
76111,pprp0fBhcMiwqe0U,LGtUPUPCs0HjZbrM
76112,pq3XcYiHrt5tlDtB,LGczYYskWtVc1rCD
76113,SLMm4hSPBChYpw4q,LGXJGZtMZr1PTV5k


In [135]:
similarity_df.to_csv("similaritys_already_searched.tsv", sep = "\t")